In [65]:
# importing the dependencies

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [67]:
data = pd.read_csv("Crop_recommendation.csv")
rain = pd.read_csv("rainfall.csv")
price = pd.read_csv("price.csv")

In [68]:
data.head(10)


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
5,69,37,42,23.058049,83.370118,7.073454,251.055000,rice
6,69,55,38,22.708838,82.639414,5.700806,271.324860,rice
7,94,53,40,20.277744,82.894086,5.718627,241.974195,rice
8,89,54,38,24.515881,83.535216,6.685346,230.446236,rice
9,68,58,38,23.223974,83.033227,6.336254,221.209196,rice


In [69]:
rain.head(5)

,SUBDIVISION,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec
0,ANDAMAN & NICOBAR ISLANDS,1901,49.2,87.1,29.2,2.3,528.8,517.5,365.1,481.1,332.6,388.5,558.2,33.6,3373.2,136.3,560.3,1696.3,980.3
1,ANDAMAN & NICOBAR ISLANDS,1902,0.0,159.8,12.2,0.0,446.1,537.1,228.9,753.7,666.2,197.2,359.0,160.5,3520.7,159.8,458.3,2185.9,716.7
2,ANDAMAN & NICOBAR ISLANDS,1903,12.7,144.0,0.0,1.0,235.1,479.9,728.4,326.7,339.0,181.2,284.4,225.0,2957.4,156.7,236.1,1874.0,690.6
3,ANDAMAN & NICOBAR ISLANDS,1904,9.4,14.7,0.0,202.4,304.5,495.1,502.0,160.1,820.4,222.2,308.7,40.1,3079.6,24.1,506.9,1977.6,571.0
4,ANDAMAN & NICOBAR ISLANDS,1905,1.3,0.0,3.3,26.9,279.5,628.7,368.7,330.5,297.0,260.7,25.4,344.7,2566.7,1.3,309.7,1624.9,630.8


In [70]:
price.head(5)

,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price
0,Andaman and Nicobar,South Andaman,Port Blair,Amaranthus,Amaranthus,04/03/2019,6000,8000,7000
1,Andaman and Nicobar,South Andaman,Port Blair,Banana - Green,Banana - Green,04/03/2019,4500,5500,5000
2,Andaman and Nicobar,South Andaman,Port Blair,Bhindi(Ladies Finger),Bhindi,04/03/2019,6000,8000,7000
3,Andaman and Nicobar,South Andaman,Port Blair,Bitter gourd,Other,04/03/2019,6000,8000,7000
4,Andaman and Nicobar,South Andaman,Port Blair,Black pepper,Other,04/03/2019,110000,130000,120000


In [71]:
# checking the data shape
data.shape

(2200, 8)

In [72]:
# dropping duplicate
data.drop_duplicates()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [73]:
# checking if there are null values
data.isnull().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [74]:
data.describe()

,N,P,K,temperature,humidity,ph,rainfall
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,25.616244,71.481779,6.469480,103.463655
std,36.917334,32.985883,50.647931,5.063749,22.263812,0.773938,54.958389
min,0.000000,5.000000,5.000000,8.825675,14.258040,3.504752,20.211267
25%,21.000000,28.000000,20.000000,22.769375,60.261953,5.971693,64.551686
50%,37.000000,51.000000,32.000000,25.598693,80.473146,6.425045,94.867624
75%,84.250000,68.000000,49.000000,28.561654,89.948771,6.923643,124.267508
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117


In [75]:
import plotly.express as px

In [76]:
fig = px.box(data, y="N", points="all")
fig.show()

In [77]:
fig = px.box(data, y="P", points="all")
fig.show()

In [78]:
fig = px.box(data, y="K", points="all")
fig.show()

In [79]:
fig = px.box(rain, y="ANNUAL", points="all")
fig.show()

In [80]:
# removing all outliers

In [81]:
"""
df_boston = data
df_boston.columns = df_boston.columns
df_boston.head()

Q1 = np.percentile(df_boston['rainfall'], 25, interpolation = 'midpoint')
Q3 = np.percentile(df_boston['rainfall'], 75, interpolation = 'midpoint')

IQR = Q3 - Q1

print("old Shape: ", df_boston.shape)

upper = np.where(df_boston['rainfall'] >= (Q3+1.5*IQR))
lower = np.where(df_boston['rainfall'] <= (Q1 - 1.5*IQR))

df_boston.drop(upper[0], inplace = True)
df_boston.drop(lower[0], inplace = True)

print("New shape: ", df_boston.shape)
"""

'\ndf_boston = data\ndf_boston.columns = df_boston.columns\ndf_boston.head()\n\nQ1 = np.percentile(df_boston[\'rainfall\'], 25, interpolation = \'midpoint\')\nQ3 = np.percentile(df_boston[\'rainfall\'], 75, interpolation = \'midpoint\')\n\nIQR = Q3 - Q1\n\nprint("old Shape: ", df_boston.shape)\n\nupper = np.where(df_boston[\'rainfall\'] >= (Q3+1.5*IQR))\nlower = np.where(df_boston[\'rainfall\'] <= (Q1 - 1.5*IQR))\n\ndf_boston.drop(upper[0], inplace = True)\ndf_boston.drop(lower[0], inplace = True)\n\nprint("New shape: ", df_boston.shape)\n'

In [82]:
def del_out(data1, str):
    df_boston = data1
    df_boston.columns = df_boston.columns
    df_boston.head()

    Q1 = np.percentile(df_boston[str], 25, interpolation = 'midpoint')
    Q3 = np.percentile(df_boston[str], 75, interpolation = 'midpoint')

    IQR = Q3 - Q1
    
    print("old Shape: ", df_boston.shape)

    upper = np.where(df_boston[str] >= (Q3+1.5*IQR))
    lower = np.where(df_boston[str] <= (Q1 - 1.5*IQR))

    df_boston.drop(upper[0], inplace = True)
    df_boston.drop(lower[0], inplace = True)

    print("New shape: ", df_boston.shape)
    return df_boston

In [83]:
data = del_out(data, "rainfall")
data = del_out(data, "K")


old Shape:  (2200, 8)
New shape:  (2101, 8)
old Shape:  (2101, 8)
New shape:  (1901, 8)


In [ ]:
# finding the outliers for rainfall dataset